In [54]:
import imaplib
import clipboard
import email
import pandas as pd
import pyautogui
import re
import time
import smtplib
import yaml
from email.mime.text import MIMEText

In [ ]:
with open("credentials.yml") as f:
    content = f.read()
    
# from credentials.yml import user name and password
my_credentials = yaml.load(content, Loader=yaml.FullLoader)


### data extraction from gmail

In [ ]:
def get_detail(gmail,password):

        # storing details
        Gmail_detail={'Subject':[],'From':[],'Body':[]}
        Subject=[]
        From=[]
        Body=[]

        # Connect to the IMAP server
        imap_server = imaplib.IMAP4_SSL('imap.gmail.com')

        # Login to the server
        imap_server.login(gmail, password)

        # Select the INBOX mailbox
        imap_server.select('INBOX')

        # Search for the latest emails in the INBOX
        status, email_ids = imap_server.search(None, 'ALL')
        latest_email_ids = email_ids[0].split()[-1:]

        # Iterate over the latest email IDs and fetch the email content
        for email_id in latest_email_ids:
            status, email_data = imap_server.fetch(email_id, '(RFC822)')
            raw_email = email_data[0][1]
            email_message = email.message_from_bytes(raw_email)
            email_subject = email_message['Subject']
            email_sender = email.utils.parseaddr(email_message['From'])[1]

            From.append(email_sender)
            Subject.append(email_subject)

            # Check if the email has multiple parts (e.g., HTML and plain text)
            if email_message.is_multipart():
                for part in email_message.walk():
                    # Process text/plain parts
                    if part.get_content_type() == 'text/plain':
                        email_body = part.get_payload(decode=True)


                        Body.append(email_body.decode())


            else:
                # Process single part email (no multipart)
                email_body = email_message.get_payload(decode=True)
                Body.append(email_body.decode())


        # making dict
        Gmail_detail['From']=From
        Gmail_detail['Subject']=Subject
        Gmail_detail['Body']=Body

        # Close the IMAP connection
        imap_server.logout()
        return Gmail_detail


In [ ]:
def extract_data_from_text(text):

      # Extracting the topic
      topic_match = re.search(r"Topic - (.+)", text)
      topic = topic_match.group(1).strip() if topic_match else None

      # Extracting the time
      time_match = re.search(r"Time - (.+)", text)
      time = time_match.group(1).strip() if time_match else None

      # Extracting the type
      type_match = re.search(r"Type - (.+)", text)
      type = type_match.group(1).strip() if type_match else None

      # Extracting the host
      host_match = re.search(r"Host - (.+)", text)
      host = host_match.group(1).strip() if host_match else None


      # extracted information
      extracted_res={'topic':'','time':'','type':'','host':''}
      extracted_res['topic']=topic
      extracted_res['time']=time
      extracted_res['type']=type
      extracted_res['host']=host

      return extracted_res

In [ ]:
result=get_detail(my_credentials["user"],my_credentials["password"])

text=result['Body'][0]
res={}

res=extract_data_from_text(text)
res['Subject']=result['Subject']
res['From']=result['From']

print(res)

### Zoom meeting creating


In [58]:
import pyautogui
import time
pyautogui.press('win')  # Press the Windows key
time.sleep(2)
pyautogui.typewrite('zoom') # Type 'notepad' to search for the Notepad application

time.sleep(2)
pyautogui.press('enter')  # Press Enter to open Notepad
time.sleep(2)  # Wait for Notepad to open

# clicking for scheduling
button7location = pyautogui.locateOnScreen('schedule.png') # returns (left, top, width, height) of matching region
button7location 
pyautogui.click(pyautogui.moveTo(button7location))
time.sleep(6)
# writting data
topic=res["topic"]
host=res["host"]
start_time=res["time"]
end_time=''

person_name="share"
def type_slowly(word):
    for character in word:
        pyautogui.write(character)
        time.sleep(0.1)

        
type_slowly(topic)

pyautogui.press('tab')
pyautogui.press('tab')
type_slowly(host)
time.sleep(1)

pyautogui.press('tab')
pyautogui.press('enter')
pyautogui.press('tab')
pyautogui.press('tab')

# type_slowly(start_date)
pyautogui.press('tab')
# pyautogui.press('tab')

type_slowly(start_time)
time.sleep(1)
pyautogui.press('tab')


# type_slowly(end_time)
time.sleep(1)
pyautogui.press('tab')


# type_slowly(end_time)
# pyautogui.press('tab')
# pyautogui.press('tab')

# after end_date
pyautogui.press('tab')
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('enter')
time.sleep(1)

# after generative
pyautogui.press('tab')
# type_slowly(passcode)
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('enter')

button7location = pyautogui.locateOnScreen('save.png') # returns (left, top, width, height) of matching region
button7location 
pyautogui.click(pyautogui.moveTo(button7location))

time.sleep(2)
button7location = pyautogui.locateOnScreen('copy_link.png') # returns (left, top, width, height) of matching region
button7location 
pyautogui.click(pyautogui.moveTo(button7location))
pyautogui.hotkey('alt', 'f4')
pyautogui.hotkey('alt', 'f4')
pyautogui.hotkey('ctrl', 'v')
link =clipboard.paste()
print(type(link))
print(link)


<class 'str'>
Baburam Yadav is inviting you to a scheduled Zoom meeting.

Topic: Talk on RPA
Time: Jun 25, 2023 12:40 PM India

Join Zoom Meeting
https://us05web.zoom.us/j/83664193626?pwd=S2ZFTDRYUHBHWDlNc3dOQzJqYmxiUT09

Meeting ID: 836 6419 3626
Passcode: 9iU7s4




### Code for sending mail

In [59]:


# Create a MIME text object with the email content
message = MIMEText(link)

# Set the sender and recipient addresses
FROM = my_credentials["user"]
message["From"] = FROM
message["To"] = my_credentials["to"]
message["Subject"] = "Zoom meeting"

# Connect to the SMTP server
smtp_server = "smtp.gmail.com"
smtp_port = 587
smtp_password = my_credentials["password"]

with smtplib.SMTP(smtp_server, smtp_port) as server:
    # Start the secure connection if using TLS
    server.starttls()

    # Login to the SMTP server if authentication is required
    server.login(FROM, smtp_password)

    # Send the email
    server.send_message(message)

    # Close the connection
    print("message sent successfully")
    server.quit()


message sent successfully
